# SA-CCR Euler allocation of an exemplary multi asset class portfolio

We want to show at an example, how the Euler allocation of trades can be summed up to subportfolios to still accurately represent the risk dynamics of the allocated risk measure.

We combine the exemplary equity portfolio (just the Adidas options) with the exemplary rates portfolio set up in the previous sections within a joint portfolio.

In [1]:
import QuantLib as ql
from IPython.core.display import display, Markdown
from allocation.Enums import FdApproach2
from allocation.eulerAllocator import EulerAllocator
from collateralAgreement.collateralAgreement import CollateralAgreement, InitialMargining, Margining
from instruments.equity_instruments.equityOption import EquityOption
from instruments.interestRateInstrument.irs import IRS
from instruments.interestRateInstrument.swaption import Swaption
from jupyterUtils import export
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import SwapDirection, Stock, TradeType, TradeDirection, AssetClass
import pandas as pd

asdf =1

In [2]:
eq_opt_ads_call = EquityOption(underlying=Stock.ADS,
                               maturity=ql.Period(1, ql.Years),
                               notional=2000000,
                               tradeType=TradeType.CALL,
                               tradeDirection=TradeDirection.LONG)

eq_opt_ads_put = EquityOption(underlying=Stock.ADS,
                              maturity=ql.Period(1, ql.Years),
                              notional=3000000,
                              tradeType=TradeType.PUT,
                              tradeDirection=TradeDirection.LONG)

payer_usd_6Y = IRS(notional = 1000000000,
                   swapDirection=SwapDirection.PAYER,
                   timeToSwapStart=ql.Period(2, ql.Days),
                   timeToSwapEnd=ql.Period(6, ql.Years),
                   index = InterestRateIndex.USDLIBOR3M)

receiver_eur_6Y = IRS(notional=180000000,
                      swapDirection=SwapDirection.RECEIVER,
                      timeToSwapStart=ql.Period(2,ql.Days),
                      timeToSwapEnd=ql.Period(6,ql.Years),
                      index=InterestRateIndex.EURIBOR6M)

ul_swap = IRS(notional=500000000,
              swapDirection=SwapDirection.RECEIVER,
              timeToSwapStart=ql.Period(1, ql.Years),
              timeToSwapEnd=ql.Period(6, ql.Years),
              index = InterestRateIndex.USDLIBOR3M)
rec_swaption_1_6 = Swaption(underlyingSwap=ul_swap,
                            optionMaturity=ql.Period(1, ql.Years))

In [3]:
trades = [eq_opt_ads_call,
          eq_opt_ads_put,
          payer_usd_6Y,
          receiver_eur_6Y,
          rec_swaption_1_6]

joint_ca_vm_only = CollateralAgreement(margining=Margining.MARGINED,
                                       initialMargining=InitialMargining.NO_IM)
joint_ca_vm_only.link_sa_ccr_instance(SA_CCR(joint_ca_vm_only))
joint_ca_vm_only.add_trades(trades)

joint_ca_vm_and_im = CollateralAgreement(margining=Margining.MARGINED,
                                         initialMargining=InitialMargining.SIMM)
joint_ca_vm_and_im.link_sa_ccr_instance(SA_CCR(joint_ca_vm_and_im))
joint_ca_vm_and_im.add_trades(trades)

And perform an Euler allocation using a central difference approach.

In [4]:
eulerAllocator_vm = EulerAllocator(joint_ca_vm_only)
eulerAllocator_vm.fdApproach2 = FdApproach2.Central
eulerAllocator_vm_and_im = EulerAllocator(joint_ca_vm_and_im)
eulerAllocator_vm_and_im.fdApproach2 = FdApproach2.Central

alloc_ead_under_vm = eulerAllocator_vm.allocate_ead()
alloc_ead_under_vm_and_im = eulerAllocator_vm_and_im.allocate_ead()
alloc_im = eulerAllocator_vm_and_im.allocate_im()

ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /crifs (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A5B7295CD0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

We then display the results on a trade level:

In [ ]:
def trrep(t):
    import ast
    if t.assetClass == AssetClass.IR:
        dic = ast.literal_eval(str(t))
        if dic['Instrument'] in ['IRS','OIS']:
            return '{:.0f}Mn '.format(dic['Notional']/1000000)+t.index.name+' '+t.swapDirection.name+' '+dic['Instrument']+' '+str(t.ql_timeToSwapEnd)
        else:
            return '{:.0f}Mn '.format(t.underlying_swap.notional/1000000)+t.underlying_swap.index.name+' '+t.underlying_swap.swapDirection.name+' '+dic['Instrument']+' '+str(t.underlying_swap.ql_timeToSwapStart) + ' to ' + str(t.underlying_swap.ql_timeToSwapEnd)
    else:
        dic = ast.literal_eval(str(t))
        return '{:.0f}Mn '.format(dic['Notional']/1000000)+t.underlying.name+' '+dic['TradeType']+' ' +str(t.ql_maturity)

In [ ]:
pd.options.display.float_format = '{:,.0f} USD'.format
data = {'Title':[trrep(eq_opt_ads_call),
                 trrep(eq_opt_ads_put),
                 trrep(receiver_eur_6Y),
                 trrep(payer_usd_6Y),
                 trrep(rec_swaption_1_6),
                 'Overall Portfolio'],
        'Allocated EAD VM only':[alloc_ead_under_vm[eq_opt_ads_call],
                                 alloc_ead_under_vm[eq_opt_ads_put],
                                 alloc_ead_under_vm[receiver_eur_6Y],
                                 alloc_ead_under_vm[payer_usd_6Y],
                                 alloc_ead_under_vm[rec_swaption_1_6],
                                 sum(alloc_ead_under_vm.values())],
        'Allocated IM':[alloc_im[eq_opt_ads_call],
                        alloc_im[eq_opt_ads_put],
                        alloc_im[receiver_eur_6Y],
                        alloc_im[payer_usd_6Y],
                        alloc_im[rec_swaption_1_6],
                        sum(alloc_im.values())],
        'Allocated EAD VM + IM':[alloc_ead_under_vm_and_im[eq_opt_ads_call],
                                 alloc_ead_under_vm_and_im[eq_opt_ads_put],
                                 alloc_ead_under_vm_and_im[receiver_eur_6Y],
                                 alloc_ead_under_vm_and_im[payer_usd_6Y],
                                 alloc_ead_under_vm_and_im[rec_swaption_1_6],
                                 sum(alloc_ead_under_vm_and_im.values())]}
df = pd.DataFrame.from_dict(data)
df = df.set_index('Title')
df.index.name = None
df

We can aggregate the results across the asset classes.

In [ ]:
sum([alloc_ead_under_vm[t] for t in alloc_ead_under_vm.keys() if t.assetClass == AssetClass.EQ])

In [ ]:
pd.options.display.float_format = '{:,.0f} USD'.format
data = {'Title':['Equity Subportfolio',
                 'Rates Subportfolio'],
        'Allocated EAD VM only':[sum([alloc_ead_under_vm[t] for t in alloc_ead_under_vm.keys() if t.assetClass == AssetClass.EQ]),
                                 sum([alloc_ead_under_vm[t] for t in alloc_ead_under_vm.keys() if t.assetClass == AssetClass.IR])],
        'Allocated IM':[sum([alloc_im[t] for t in alloc_im.keys() if t.assetClass == AssetClass.EQ]),
                        sum([alloc_im[t] for t in alloc_im.keys() if t.assetClass == AssetClass.IR])],
        'Allocated EAD VM + IM':[sum([alloc_ead_under_vm_and_im[t] for t in alloc_ead_under_vm_and_im.keys() if t.assetClass == AssetClass.EQ]),
                        sum([alloc_ead_under_vm_and_im[t] for t in alloc_ead_under_vm_and_im.keys() if t.assetClass == AssetClass.IR])]
        }
df = pd.DataFrame.from_dict(data)
df = df.set_index('Title')
df.index.name = None
df


This result is discussed further in section \todo{Reference required}.

We can aggregate across the asset classes to confirm that no hedges between the asset classes take place as the results are the same as in sections \ref{sa-ccr-euler-allocation-of-exemplary-equity-portfolio} and \ref{euler-allocation-of-an-exemplary-rates-portfolio}.

Subportfolio allocations can also be calculated directly by calculating partial derivatives against an entire subportfolio instead of a single trade which saves computational effort.

Using a central difference approach we first calculate the partial derivative of the EAD under VM and IM of the equity subportfolio:

In [ ]:
bump = 0.00005

joint_ca_vm_and_im.replace_trade(
    [eq_opt_ads_put,
     eq_opt_ads_call],
    [eq_opt_ads_put.get_bumped_copy(rel_bump_size=-bump),
     eq_opt_ads_call.get_bumped_copy(rel_bump_size=-bump),])

bump_down_ead = joint_ca_vm_and_im.get_sa_ccr_model().get_ead()

joint_ca_vm_and_im.remove_all_trades()
joint_ca_vm_and_im.add_trades(trades)
joint_ca_vm_and_im.replace_trade(
    [eq_opt_ads_put,
     eq_opt_ads_call],
    [eq_opt_ads_put.get_bumped_copy(rel_bump_size=bump),
     eq_opt_ads_call.get_bumped_copy(rel_bump_size=bump),])

bump_up_ead = joint_ca_vm_and_im.get_sa_ccr_model().get_ead()

eq_pf_central_difference = (bump_up_ead-bump_down_ead)/(2*bump)

In [ ]:
f'{eq_pf_central_difference:,.0f} USD'

In [ ]:
joint_ca_vm_and_im.remove_all_trades()
joint_ca_vm_and_im.add_trades(trades)
joint_ca_vm_and_im.replace_trade(
    [receiver_eur_6Y,
     payer_usd_6Y,
     rec_swaption_1_6],
    [receiver_eur_6Y.get_bumped_copy(rel_bump_size=-bump),
     payer_usd_6Y.get_bumped_copy(rel_bump_size=-bump),
     rec_swaption_1_6.get_bumped_copy(rel_bump_size=-bump),])

bump_down_ead = joint_ca_vm_and_im.get_sa_ccr_model().get_ead()

joint_ca_vm_and_im.remove_all_trades()
joint_ca_vm_and_im.add_trades(trades)
joint_ca_vm_and_im.replace_trade(
    [receiver_eur_6Y,
     payer_usd_6Y,
     rec_swaption_1_6],
    [receiver_eur_6Y.get_bumped_copy(rel_bump_size=bump),
     payer_usd_6Y.get_bumped_copy(rel_bump_size=bump),
     rec_swaption_1_6.get_bumped_copy(rel_bump_size=bump),])

bump_up_ead = joint_ca_vm_and_im.get_sa_ccr_model().get_ead()

ir_pf_central_difference = (bump_up_ead-bump_down_ead)/(2*bump)

In [ ]:
f'{ir_pf_central_difference:,.0f} USD'

In [ ]:
display(Markdown('When bumping all trades of a subportfolio simultaneously we yield the same results of {:,.0f} USD for the equity subportfolio and {:,.0f} USD for the rates subportfolio whilst performing less recalculations of the portfolio.'.format(eq_pf_central_difference, ir_pf_central_difference)))

In [ ]:
export("SA-CCR Euler allocation of an examplary multi asset class portfolio.ipynb")